In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [2]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [3]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=2,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 32, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 64, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 128, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 256, num_blocks[3], stride=2)
        self.linear = nn.Linear(256*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [4]:
model = ResNet(BasicBlock, [5,1,1,4])

In [5]:
# Set device to use
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Instantiate ResNet model
model = model.to(device)

# Generate example input data
inputs = torch.randn(1, 3, 64, 64).to(device)

# Print model summary
summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 33, 33]             768
       BatchNorm2d-2           [-1, 64, 33, 33]             128
            Conv2d-3           [-1, 32, 33, 33]          18,432
       BatchNorm2d-4           [-1, 32, 33, 33]              64
            Conv2d-5           [-1, 32, 33, 33]           9,216
       BatchNorm2d-6           [-1, 32, 33, 33]              64
            Conv2d-7           [-1, 32, 33, 33]           2,048
       BatchNorm2d-8           [-1, 32, 33, 33]              64
        BasicBlock-9           [-1, 32, 33, 33]               0
           Conv2d-10           [-1, 32, 33, 33]           9,216
      BatchNorm2d-11           [-1, 32, 33, 33]              64
           Conv2d-12           [-1, 32, 33, 33]           9,216
      BatchNorm2d-13           [-1, 32, 33, 33]              64
       BasicBlock-14           [-1, 32,

In [6]:
# define data augmentation transforms
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.2010])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.2010])
])

# load CIFAR10 train and test datasets
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

100%|██████████| 170498071/170498071 [00:13<00:00, 12987730.27it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
import torch.optim as optim
import torchvision

In [8]:
num_epochs = 30
batch_size = 128
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
transform = transforms.Compose(
    [transforms.RandomCrop(32, padding=4),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        log_interval = 100

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

    # Evaluate the model on the test set
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item() * data.size(0)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.808073
Train Epoch: 0 [12800/50000 (26%)]	Loss: 1.480679
Train Epoch: 0 [25600/50000 (51%)]	Loss: 1.378804
Train Epoch: 0 [38400/50000 (77%)]	Loss: 1.366213
Test set: Average loss: 1.2334, Accuracy: 5608/10000 (56%)

Train Epoch: 1 [0/50000 (0%)]	Loss: 1.066705
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.004192
Train Epoch: 1 [25600/50000 (51%)]	Loss: 0.865692
Train Epoch: 1 [38400/50000 (77%)]	Loss: 0.872037
Test set: Average loss: 0.9721, Accuracy: 6668/10000 (67%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 0.734947
Train Epoch: 2 [12800/50000 (26%)]	Loss: 0.871761
Train Epoch: 2 [25600/50000 (51%)]	Loss: 0.732248
Train Epoch: 2 [38400/50000 (77%)]	Loss: 0.819324
Test set: Average loss: 0.8295, Accuracy: 7062/10000 (71%)

Train Epoch: 3 [0/50000 (0%)]	Loss: 0.560251
Train Epoch: 3 [12800/50000 (26%)]	Loss: 0.553902
Train Epoch: 3 [25600/50000 (51%)]	Loss: 0.533274
Train Epoch: 3 [38400/50000 (77%)]	Loss: 0.514162
Test set: Average loss: 0.7675, Accuracy